prepare BERT model

In [2]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

/Users/josephtey/Projects/auto-ed-coder/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


use BERT to embed pile dataset

In [3]:
inputs = tokenizer("Hello world!", return_tensors="pt")
outputs = model(**inputs)

In [4]:
from datasets import load_dataset

ds = load_dataset("JeanKaddour/minipile")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Using the latest cached version of the dataset since JeanKaddour/minipile couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/josephtey/.cache/huggingface/datasets/JeanKaddour___minipile/default/0.0.0/18ad1b0c701eaa0de03d3cecfdd769cbc70ffbd0 (last modified on Sat Jul  6 16:58:10 2024).


In [6]:
from torch.utils.data import Dataset

class MiniPileDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]
    
# Convert the dataset to MiniPileDataset format
dataset = MiniPileDataset(ds['train']['text'])

train SAE

In [7]:
import torch.optim as optim
import torch
from utils.sae import SparseAutoencoder, SparseAutoencoderConfig

# Assuming `dataset` is a PyTorch Dataset loaded and ready to use
data_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=True)

# Initialize the model
config = SparseAutoencoderConfig(d_model=768, d_sparse=8 * 768)
model = SparseAutoencoder(config)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    total_loss = 0
    for data in data_loader:
        optimizer.zero_grad()
        
        # Assuming data is already on the correct device and in the correct format
        _, _, loss, _ = model.forward(data, return_loss=True, sparsity_scale=1.0)
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print(f"Epoch {epoch+1}, Average Loss: {total_loss / len(data_loader)}")